In [13]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import shutil
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
!pip install wandb -qU
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
Mounted at /content/drive


In [14]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
root_dir = '/content/fashion_train_data'
os.makedirs(root_dir, exist_ok=True)
for i in range(10):
    os.makedirs(os.path.join(root_dir, str(i)), exist_ok=True)

In [4]:
trainset = torchvision.datasets.FashionMNIST(root=root_dir, train=True, download=True, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, batch_size=1, shuffle=True)

100%|██████████| 26421880/26421880 [00:05<00:00, 4689923.13it/s] 


Extracting /content/fashion_train_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/fashion_train_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 274818.20it/s]


Extracting /content/fashion_train_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/fashion_train_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5050273.66it/s]


Extracting /content/fashion_train_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/fashion_train_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8276073.97it/s]

Extracting /content/fashion_train_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/fashion_train_data/FashionMNIST/raw



In [5]:
class_names = trainset.classes

In [6]:
def save_images_to_folders(dataset, root_dir):
    for idx, (image, label) in enumerate(tqdm(dataset)):
        label_dir = os.path.join(root_dir, str(label))
        img_path = os.path.join(label_dir, f'{idx}.png')
        torchvision.utils.save_image(image, img_path)

In [7]:
save_images_to_folders(trainset, root_dir)

100%|██████████| 60000/60000 [00:41<00:00, 1454.68it/s]


In [8]:
shutil.rmtree('/content/fashion_train_data/FashionMNIST')

In [9]:
import os
import pandas as pd


root_dir = '/content/fashion_train_data'


data = []

for label in os.listdir(root_dir):
    label_dir = os.path.join(root_dir, label)
    if os.path.isdir(label_dir):
        for file_name in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file_name)
            if os.path.isfile(file_path):
                data.append([f"{label}/{file_name}", label])


df = pd.DataFrame(data, columns=['filepath', 'label'])


df.to_csv('annotations.csv', index=False)

print("annotations.csv file created successfully.")


annotations.csv file created successfully.


In [16]:
config = {
    "learning_rate": 0.01,
    "batch_size": 32,
    "epochs": 10,
}

In [22]:
run = wandb.init(project="fmnist-classification", name="initial_training", config=config, job_type="train")
annotations_path = '/content/annotations.csv'
labels_at = wandb.Artifact("annotations", type="dataset")
labels_at.add_reference(f'file://{annotations_path}')
run.log_artifact(labels_at)

<Artifact annotations>

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import pandas as pd
from PIL import Image

class FMNISTDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert("L")
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

In [34]:
dataset = FMNISTDataset(annotations_file='/content/annotations.csv', img_dir='/content/fashion_train_data', transform=transforms.ToTensor())
train_loader = DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)


In [35]:
class model1(nn.Module):
  def __init__(self, in_shape, hidden_units, out_shape):
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=in_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(490, out_shape)
    )

  def forward(self, x):
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))

model = model1(1, 10, 10).to(device)

In [36]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model.parameters(), lr=config['learning_rate'])

def accuracy_function(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [33]:
from tqdm.auto import tqdm
def train_function(model, device, train_dataloader, loss_function, optimizer):
  model.train()
  model = model.to(device)
  training_loss = 0
  trng_acc = 0
  for X_train, y_train in tqdm(train_dataloader):
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_train_logits = model(X_train)
    loss = loss_function(y_train_logits, y_train)
    training_loss += loss
    trng_acc += accuracy_function(y_train, torch.argmax(y_train_logits, dim=1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  # find average loss per batch
  return (training_loss/len(train_dataloader), trng_acc/len(train_dataloader))


def validate_function(model, device, validation_dataloader, loss_function, accuracy_function):
  model.eval()
  model = model.to(device)
  validation_loss = 0
  validation_acc = 0

  with torch.inference_mode():
    for X_val, y_val in tqdm(validation_dataloader):
      X_val , y_val = X_val.to(device), y_val.to(device)
      val_logits = model(X_val)
      val_pred = torch.argmax(val_logits, dim=1)
      validation_acc += accuracy_function(y_val, val_pred)
      validation_loss += loss_function(val_logits, y_val)
  return validation_loss/len(validation_dataloader), validation_acc/len(validation_dataloader)

In [37]:
epochs = config['epochs']
for epoch in range(epochs):
  train_loss, trng_acc = train_function(model, device, train_loader, loss_fn, optimizer)
  # val_loss, val_acc = validate_function(model, device, test_loader, loss_fn, accuracy_function)
  wandb.log({
        'training_loss': train_loss,
        'training_accuracy': trng_acc,
        # 'validation_loss': val_loss,
        # 'validation_accuracy': val_acc
    })
  print(f'Epoch {epoch}: Training Loss: {train_loss:.4f}, Training Accuracy: {trng_acc:.4f} ')

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 0: Training Loss: 1.1136, Training Accuracy: 59.4617 


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1: Training Loss: 0.5180, Training Accuracy: 81.2917 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 7: Training Loss: 0.3325, Training Accuracy: 88.1167 


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 8: Training Loss: 0.3234, Training Accuracy: 88.5300 


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9: Training Loss: 0.3139, Training Accuracy: 88.9283 


In [38]:
project_name = wandb.run.project
run_name = wandb.run.name
# change notebook name according to name of current notebook
notebook_path = '/content/drive/MyDrive/Colab Notebooks/fmnist_annotations.ipynb'

In [39]:
url_path = '/content/drive/MyDrive/github_url.txt'
with open(url_path, 'r') as file:
    github_url = file.read().strip()

In [ ]:
import os
import subprocess
import shutil
directory = project_name
file_name = f"{run_name}.ipynb"
file_path = os.path.join('/content/ML-Deep-Learning/Weights_Biases', directory, file_name)